In [1]:
import os

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis
output_notebook() 

Loading BokehJS ...

In [2]:
DT = 0.1 # Simulation parameter

KC = 0.16
KH = 0.0896
VF = 25.0
W = 6.25
E  = 25.0*0.3
GCAV = 1/(KC*W)
GHDV = 1/(KH*W)
SCAV = VF/(KC*W)+1/KC
SHDV = VF/(KH*W)+1/KH
dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}
U_MAX = 1.5
U_MIN = -1.5

In [3]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

try: 
    ltbstr = 'Loaded table in: '
    connection = engine.connect()    
    traj = Table('traj', metadata, autoload=True, autoload_with=engine)
    closed = Table('closed', metadata, autoload=True, autoload_with=engine)
    headway = Table('headway', metadata, autoload=True, autoload_with=engine)
    control = Table('control', metadata, autoload=True, autoload_with=engine)    
except:
    ltbstr = 'Table could not be loaded: '
finally: 
    print(ltbstr, engine)

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


In [4]:
stmt = select([closed])
print(stmt)
results = connection.execute(stmt).fetchall()

SELECT closed.ti, closed.id, closed.type, closed.tron, closed.voie, closed.dst, closed.abs, closed.vit, closed.ldr, closed.spc, closed.vld, closed.ctr 
FROM closed


In [5]:
column_names = closed.columns.keys()
trajCLDf = pd.DataFrame(results, columns = column_names)
trajCLDf.head()

,ti,id,type,tron,voie,dst,abs,vit,ldr,spc,vld,ctr
0,1.3,0,CAV,In_main,1,1.25,-998.75,25.0,0,31.25,25.0,None
1,1.4,0,CAV,In_main,1,3.75,-996.25,25.0,0,31.25,25.0,None
2,1.5,0,CAV,In_main,1,6.25,-993.75,25.0,0,31.25,25.0,None
3,1.6,0,CAV,In_main,1,8.75,-991.25,25.0,0,31.25,25.0,None
4,1.7,0,CAV,In_main,1,11.25,-988.75,25.0,0,31.25,25.0,None


In [6]:
trajCLDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5289 entries, 0 to 5288
Data columns (total 12 columns):
ti      5289 non-null float64
id      5289 non-null int64
type    5289 non-null object
tron    5289 non-null object
voie    5289 non-null int64
dst     5289 non-null float64
abs     5289 non-null float64
vit     5289 non-null float64
ldr     5289 non-null int64
spc     5289 non-null float64
vld     5289 non-null float64
ctr     0 non-null object
dtypes: float64(6), int64(3), object(3)
memory usage: 495.9+ KB


In [7]:
colormap = {'In_main': 'blue', 'In_onramp': 'red', 'Merge_zone': 'green', 'Out_main': 'yellow'}
keys = trajCLDf.id.unique()
keysstr = [str(k) for k in keys]
values =  Viridis[len(keys)]
colormapid = dict(zip(keys,values))
colormapID = {}
colors = [colormapid[x] for x in trajCLDf.id]

# Spacing 

# Bokeh 
p = figure(title = "Headspace")
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Spacing [m]'

p.circle(trajCLDf['ti'], trajCLDf['spc'], color = colors, size = 2)
show(p)

#
#trajDf.plot(kind='scatter',x='ti',y ='spc', c = trajDf.tron.map(colormap), figsize = (7.5, 7.5), s = 2, grid = True);

In [8]:
stmt = select([headway])
print(stmt)
results = connection.execute(stmt).fetchall()

SELECT headway.ti, headway.id, headway.gapt 
FROM headway


In [9]:
column_names = headway.columns.keys()
headDf = pd.DataFrame(results, columns = column_names)
headDf.head()

,ti,id,gapt
0,1.3,0,1.0
1,1.4,0,1.0
2,1.5,0,1.0
3,1.6,0,1.0
4,1.7,0,1.0


In [10]:
# Bokeh
p = figure(title = "Time Gap")
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Tau [s]'

p.circle(trajCLDf['ti'], (trajCLDf['spc']-1/KC)/trajCLDf['vit'] , color = colors, size = 2)
p.circle(headDf.ti, headDf.gapt, size = 1,color='lightblue')

show(p)

#
#trajDf.plo

In [11]:
stmt = select([control])
print(stmt)
results = connection.execute(stmt).fetchall()

SELECT control.ti, control.id, control.type, control.tron, control.voie, control.ctr 
FROM control


In [12]:
column_names = control.columns.keys()
controlDf = pd.DataFrame(results, columns = column_names)
controlDf.head()

,ti,id,type,tron,voie,ctr
0,1.3,0,CAV,In_main,1,0.0
1,1.4,0,CAV,In_main,1,0.0
2,1.5,0,CAV,In_main,1,0.0
3,1.6,0,CAV,In_main,1,0.0
4,1.7,0,CAV,In_main,1,0.0


In [13]:
controlDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5274 entries, 0 to 5273
Data columns (total 6 columns):
ti      5274 non-null float64
id      5274 non-null int64
type    5274 non-null object
tron    5274 non-null object
voie    5274 non-null int64
ctr     5274 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 247.3+ KB


In [14]:
colormap = {'In_main': 'blue', 'In_onramp': 'red', 'Merge_zone': 'green', 'Out_main': 'yellow'}
keys = trajCLDf.id.unique()
keysstr = [str(k) for k in keys]
values =  Viridis[len(keys)]
colormapid = dict(zip(keys,values))
colormapID = {}
colors = [colormapid[x] for x in controlDf.id]

# Control 

# Bokeh 
p = figure(title = "Control signal")
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Acceleration [m/s^2]'

p.circle(controlDf['ti'], controlDf['ctr'], color = colors, size = 2)
show(p)

#
#trajDf.plot(kind='scatter',x='ti',y ='spc', c = trajDf.tron.map(colormap), figsize = (7.5, 7.5), s = 2, grid = True);

In [15]:
p = figure(title = "Trajectories")
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Position [m]'
colors = [colormapid[x] for x in trajCLDf.id]


p = figure(title = "Trajectories")
p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'X [m]'
p.circle(trajCLDf['ti'], trajCLDf['abs'], color = colors, size = 2)
show(p)
